¿Cuál es la acción más realizada por usuarios que no están registrados? (⭐⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

E: Invalid operation update]
openjdk-8-jdk-headless is already the newest version (8u372-ga~us1-0ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_logs = sqlContext.read.csv('/content/drive/MyDrive/logs.csv', header=True, inferSchema=True, multiLine=True, escape='"')
rdd_logs = df_logs.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Por enunciado, los usuarios cons ips si o si no estan logeados, los usarios que no tengan ip ni usario no los considero

Elimino las columnas que no uso

In [ ]:
rdd_filtrado = rdd_logs.map(lambda x: (x.contributor_ip, x.action))

In [ ]:
rdd_anonimos = rdd_filtrado.filter(lambda x: x[0] != None)

Necesito agrupar las acciones y contar cuantos usuarios las realizan

In [ ]:
rdd_anonimos = rdd_anonimos.map(lambda x: (x[1], 1))

Veo por acciones la cantidad de usuarios no registrados que las realizaron.

In [ ]:
acciones_contadas_rdd = rdd_anonimos.reduceByKey(lambda x,y: x + y)

Doy la accion mas realizada por los usarios no registrados.

In [ ]:
accion_mas_realizada =  acciones_contadas_rdd.reduce(lambda x, y: x if x[1] > y[1] else y)

In [ ]:
accion_mas_realizada

('create', 124119)

In [ ]:
rdd_logs.take(1)

[Row(item_id=1, timestamp=datetime.datetime(2004, 12, 23, 7, 10, 9), contributor_username='Sabbut', contributor_id=2075, contributor_ip=None, comment="el contenido era: 'Si aprendés Pensamiento Científico ... TE VA S AENTERAR'", logtype='delete', action='delete', title='Verificacionismo')]

In [ ]:
rdd_prueba = rdd_logs.map(lambda x: (x.item_id, 1))

In [ ]:
rdd_prueba.map(lambda x: x[0]).sum()

98059537556785

In [ ]:
rdd_prueba.map(lambda x: x[0]).reduce(lambda a, b: a + b)

98059537556785

In [ ]:
rdd_prueba.reduce(lambda x, y: x if x[0] > y[0] else y)

(15587399, 1)

In [ ]:
rdd_prueba.take(5)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
rdd_filtrado.take(10)

NameError: ignored